In [12]:
from google.colab import drive
drive.mount('/content/drive')
ROOT_PATH = '/content/drive/My Drive/capstone5703/GALIP'


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [13]:
%cd /content/drive/My Drive/capstone5703/GALIP
!pip install -r requirements.txt

/content/drive/My Drive/capstone5703/GALIP


In [14]:
!pip install ftfy regex tqdm
!pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-uny9yh06
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-uny9yh06
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done


In [15]:
!pip install "scipy<1.11.2"

In [16]:
%cd /content/drive/My Drive/capstone5703/GALIP/code
!bash scripts/train.sh /content/drive/MyDrive/capstone5703/GALIP/code/cfg/movie.yml


/content/drive/My Drive/capstone5703/GALIP/code
2024-04-29 05:30:36.073204: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-29 05:30:36.073266: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-29 05:30:36.074638: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-04-29 05:30:37.393502: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Load filenames from: ../data/movie/train/filenames.pickle (549)
Load filenames from: ../data/movie/test/filenames.pickle (138)
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloade

In [ ]:
!bash scripts/test.sh /content/drive/MyDrive/capstone5703/GALIP/code/cfg/birds.yml

sample

In [ ]:
import torch
import os
from PIL import Image
import clip
import os.path as osp
import os, sys
import torchvision.utils as vutils
sys.path.insert(0, '/content/drive/MyDrive/capstone5703/GALIP/code')

from lib.utils import load_model_weights,mkdir_p
from models.GALIP import NetG, CLIP_TXT_ENCODER

device = 'cpu' # 'cpu' # 'cuda:0'
CLIP_text = "ViT-B/32"
clip_model, preprocess = clip.load("ViT-B/32", device=device)
clip_model = clip_model.eval()

text_encoder = CLIP_TXT_ENCODER(clip_model).to(device)
netG = NetG(64, 100, 512, 256, 3, False, clip_model).to(device)
path = '/content/drive/MyDrive/capstone5703/GALIP/code/saved_models/bird/GALIP_nf64_gpu8MP_True_bird_256_2024_04_07_05_11_00/state_epoch_004.pth'
# path = 'saved_models/pretrained/pre_cc12m.pth'

checkpoint = torch.load(path, map_location=torch.device('cpu'))
netG = load_model_weights(netG, checkpoint['model']['netG'], multi_gpus=False)

batch_size = 8
noise = torch.randn((batch_size, 100)).to(device)


captions = ['a bright red head and a white body. red headed bird with large black eyes, a pasty white belly and an all black backside.']

In [ ]:
mkdir_p('./samples')

In [ ]:
# generate from text
with torch.no_grad():
    for i in range(len(captions)):
        caption = captions[i]
        tokenized_text = clip.tokenize([caption]).to(device)
        sent_emb, word_emb = text_encoder(tokenized_text)
        sent_emb = sent_emb.repeat(batch_size,1)
        fake_imgs = netG(noise,sent_emb,eval=True).float()
        name = f'{captions[i].replace(" ", "-")}'
        vutils.save_image(fake_imgs.data, 'samples/%s.png'%(name), nrow=8, value_range=(-1, 1), normalize=True)